In [110]:
import requests
import pandas as pd
from dags.enrich import enrich
from dags.transforms import clean_dataframe

r = requests.get(url='http://65.108.50.112:5000/data/arxiv')
df = pd.json_normalize(r.json(), ["result"])

In [111]:
df = clean_dataframe(df[:5])

In [112]:
df = enrich(df)

Full name, aff and gender search:
Vanessa Casagrande
{'given': 'Vanessa', 'family': 'Casagrande', 'affiliation': None, 'gender': 'female', 'full_name': 'Vanessa Casagrande'}
Full name, aff and gender search:
Yuichi Togashi
{'given': 'Yuichi', 'family': 'Togashi', 'affiliation': None, 'gender': 'male', 'full_name': 'Yuichi Togashi'}
Full name, aff and gender search:
Alexander Mikhailov
{'given': 'Alexander S.', 'family': 'Mikhailov', 'affiliation': None, 'gender': 'male', 'full_name': 'Alexander Mikhailov'}
Full name, aff and gender search:
Simon Malham
{'given': 'Simon J. A.', 'family': 'Malham', 'affiliation': None, 'gender': 'Unknown', 'full_name': 'Simon Malham'}
Full name, aff and gender search:
Anke Wiese
{'given': 'Anke', 'family': 'Wiese', 'affiliation': None, 'gender': 'female', 'full_name': 'Anke Wiese'}
Full name, aff and gender search:
Maria Loukitcheva
{'given': 'Maria A.', 'family': 'Loukitcheva', 'affiliation': None, 'gender': 'female', 'full_name': 'Maria Loukitcheva'}
F

KeyboardInterrupt: 

In [3]:
for author in df['authors_merged'][3]:
    print(author)

{'given': 'Carlos', 'family': 'Gershenson', 'affiliation': None, 'gender': 'None', 'full_name': 'None'}


In [74]:
authors_df = df[['id','authors_merged']].copy()

In [75]:
explded = authors_df.explode("authors_merged")

In [76]:
authors_df_normalized = pd.json_normalize(explded['authors_merged'])

In [77]:
authors_df_normalized['id'] = explded['id'].tolist()

In [92]:
main = df[['published-year', 'subject','type','container-title','publisher','id','doi','title','versions','is-referenced-by-count']]

In [94]:
main

,published-year,subject,type,container-title,publisher,id,doi,title,versions,is-referenced-by-count
0,2007,Nuclear and High Energy Physics,journal-article,Physical Review D,American Physical Society (APS),0704.0001,10.1103/physrevd.76.013009,Calculation of prompt diphoton production cros...,v2,46
1,2009,Discrete Mathematics and Combinatorics,journal-article,Graphs and Combinatorics,Springer Science and Business Media LLC,0704.0002,10.1007/s00373-008-0834-4,Sparsity-certifying Graph Decompositions,v2,11
2,2021,General Medicine,journal-article,International Journal of Physics,"Science and Education Publishing Co., Ltd.",0704.0003,10.12691/ijp-8-1-3,The Evolution of the Earth-Moon System Based o...,v3,0
3,2008,Discrete Mathematics and Combinatorics,journal-article,Discrete Mathematics &amp; Theoretical Compute...,Centre pour la Communication Scientifique Dire...,0704.0004,10.46298/dmtcs.421,A determinant of Stirling cycle numbers counts...,v1,0
4,2008,General Mathematics,journal-article,Illinois Journal of Mathematics,Duke University Press,0704.0005,10.1215/ijm/1248355358,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,v1,2


In [96]:
import os
df = pd.read_csv(os.path.join('data', 'mock_main.csv'))

In [103]:
sql_statement = """INSERT INTO staging_main
(publication_year, scientific_domain, type_name, pub_venue, publisher, arxiv_ID, doi, title, latest_version_nr, citaton_count)
"""

for idx, row in df.iterrows():
    sql_statement += 'UNION ALL\n'

    sql_statement += f"""
        SELECT CAST('{row['published-year']}' AS INT) as publication_year
        , '{row['subject']}' as scientific_domain
        , '{row['type']}' as type_name
        , '{row['container-title']}' as pub_venue
        , '{row['publisher']}' as publisher
        , '{row['id']}' as arxiv_ID
        , '{row['doi']}' as doi
        , '{row['title']}' as title
        , '{row['versions']}' as latest_version_nr
        , CAST('{row['is-referenced-by-count']}' AS INT) as citaton_count
        """

    print(sql_statement)
    print()

INSERT INTO staging_main
(publication_year, scientific_domain, type_name, pub_venue, publisher, arxiv_ID, doi, title, latest_version_nr, citaton_count)

        SELECT CAST('2007' AS INT) as publication_year
        , 'Nuclear and High Energy Physics' as scientific_domain
        , 'journal-article' as type_name
        , 'Physical Review D' as pub_venue
        , 'American Physical Society (APS)' as publisher
        , '704.0001' as arxiv_ID
        , '10.1103/physrevd.76.013009' as doi
        , 'Calculation of prompt diphoton production cross sections at Fermilab Tevatron and CERN LHC energies' as title
        , 'v2' as latest_version_nr
        , CAST('46' AS INT) as citaton_count
        

INSERT INTO staging_main
(publication_year, scientific_domain, type_name, pub_venue, publisher, arxiv_ID, doi, title, latest_version_nr, citaton_count)

        SELECT CAST('2007' AS INT) as publication_year
        , 'Nuclear and High Energy Physics' as scientific_domain
        , 'journal-art

In [99]:
sql_statement

"INSERT INTO staging_main\n(publication_year, scientific_domain, type_name, pub_venue, publisher, arxiv_ID, doi, title, latest_version_nr, citaton_count)\n\n        SELECT CAST('2009' AS INT) as publication_year\n        , 'Discrete Mathematics and Combinatorics' as scientific_domain\n        , 'journal-article' as type_name\n        , 'Graphs and Combinatorics' as pub_venue\n        , 'Springer Science and Business Media LLC' as publisher\n        , '704.0002' as arxiv_ID\n        , '10.1007/s00373-008-0834-4' as doi\n        , 'Sparsity-certifying Graph Decompositions' as title\n        , 'v2' as latest_version_nr\n        , CAST('11' AS INT) as citaton_count\n        UNION ALL\n\n        SELECT CAST('2021' AS INT) as publication_year\n        , 'General Medicine' as scientific_domain\n        , 'journal-article' as type_name\n        , 'International Journal of Physics' as pub_venue\n        , 'Science and Education Publishing Co., Ltd.' as publisher\n        , '704.0003' as arxiv_I

In [106]:
authors_df_normalized = pd.read_csv(os.path.join('data', 'mock_authors.csv'))
authors_df_normalized

,given,family,affiliation,gender,full_name,id
0,C.,Balázs,Gragra Ülikool,male,Caleb Balázs,704.0001
1,E. L.,Berger,Prapa Ülikool,female,Edna Berger,704.0001
2,P. M.,Nadolsky,Narva Kolledž,male,Pavel Nadolsky,704.0001
3,C. -P.,Yuan,Huru University,male,Cole Yuan,704.0001
4,Ileana,Streinu,Grando University,female,Ileana Streinu,704.0002
5,Louis,Theran,Harvard University,male,Louis Theran,704.0002
6,Hongjun,Pan,Huange University,male,Hongjun Pan,704.0003
7,David,Callan,Malgo School,male,David Callan,704.0004
8,Wael,Abu-Shammala,Ropero University,male,Wael Abu-Shammala,704.0005
9,Alberto,Torchinsky,Google,male,Alberto Torchinsky,704.0005


In [107]:
import scholarmetrics

In [109]:
scholarmetrics.hindex([9,6,5])

3